### Render stac outputs with mujoco

In [1]:
%load_ext autoreload
%autoreload 2

# glfw is faster, but use osmesa if glfw not available
%env MUJOCO_GL=osmesa
%env PYOPENGL_PLATFORM=osmesa

import stac_mjx

import mediapy as media
from pathlib import Path
import os


# Disable jax's memory preallocation if you're running multiple notebooks using jax
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false" 

base_path = Path.cwd().parent
stac_config_path = base_path / "configs/stac.yaml"
model_config_path = base_path / "configs/rodent.yaml"

stac_cfg, model_cfg = stac_mjx.load_configs(stac_config_path, model_config_path)

env: MUJOCO_GL=osmesa
env: PYOPENGL_PLATFORM=osmesa


In [2]:
# set args
data_path = base_path / "fit_mouse_3600_newtail.p"
n_frames = 1
save_path = base_path / "videos/direct_render_skel_only_newtail.mp4"

# Call mujoco_viz
frames = stac_mjx.viz_stac(data_path, stac_cfg, model_cfg, n_frames, save_path, start_frame=0, camera="center_of_mass", base_path=Path.cwd().parent)

# Show the video in the notebook (it is also saved to the save_path)
media.show_video(frames, fps=model_cfg["RENDER_FPS"])

0it [00:00, ?it/s]/root/anaconda3/envs/stac-mjx-env/lib/python3.11/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
1it [00:00,  3.09it/s]
